In [27]:
from basic import annotation, made_utils
from importlib import reload

from collections import defaultdict

from basic.annotation import *
reload(made_utils)

<module 'basic.made_utils' from 'C:\\Users\\u0752374\\projects\\MADE_relations\\basic\\made_utils.py'>

In [21]:
reader = made_utils.TextAndBioCParser()

In [72]:
docs = reader.read_texts_and_xmls(num_docs=-1)

0/876
25/876
50/876
75/876
100/876
125/876
150/876
175/876
200/876
225/876
250/876
275/876
300/876
325/876
350/876
375/876
400/876
425/876
450/876
475/876
500/876
525/876
550/876
575/876
600/876
625/876
650/876
675/876
700/876
725/876
750/876
775/876
800/876
825/876
850/876
875/876


How many documents, entities, and relations?

In [106]:
info = defaultdict(int)
entity_types = defaultdict(int)
relation_types = defaultdict(int)

type_relations = defaultdict(int) # entity => entity
type_has_out_edge_to = defaultdict(dict)

for i, doc in enumerate(docs.values()):
    if i % 100 == 0:
        print(i)
    # How many documents, entities, and relations?
    info['num_docs'] += 1
    info['num_entities'] += len(doc.bioc_annotations)
    info['num_relations'] += len(doc.bioc_relations)
    
    # How many distinct type -> type relations?
    for relation in doc.relations:
        type1, type2 = relation.entity_types
        entity_types[type1] += 1
        entity_types[type2] += 1
      
        type_relations[' => '.join((type1, type2))] += 1
        
        if type2 in type_has_out_edge_to[type1]:
            type_has_out_edge_to[type1][type2] += 1
        else:
            type_has_out_edge_to[type1][type2] = 1
            
info['num_entity_types'] = len(entity_types)
info['num_relationship_types'] = len(type_relations)
# What types are never connected?

types_not_connected = []

for type1 in entity_types:
    for type2 in entity_types:
        if (type1, type2) not in type_relations:
            types_not_connected.append(' => '.join((type1, type2)))
        if (type2, type1) not in type_relations:
            types_not_connected.append(' => '.join((type2, type1)))

print(info)
print(type_relations)
print(type_has_out_edge_to)

0
100
200
300
400
500
600
700
800
defaultdict(<class 'int'>, {'num_docs': 876, 'num_entities': 67781, 'num_relations': 23165, 'num_entity_types': 9, 'num_relationship_types': 17})
defaultdict(<class 'int'>, {'Drug => Route': 2544, 'Drug => Indication': 4530, 'SSLIF => Severity': 2909, 'Drug => Dose': 5150, 'Drug => Frequency': 4407, 'Drug => Duration': 901, 'Drug => ADE': 2055, 'ADE => Severity': 282, 'Dose => Dose': 27, 'Indication => Severity': 269, 'Indication => Indication': 24, 'Severity => Severity': 16, 'ADE => ADE': 23, 'Duration => Duration': 5, 'Route => Route': 7, 'SSLIF => ADE': 4, 'Frequency => Frequency': 12})
defaultdict(<class 'dict'>, {'Drug': {'Route': 2544, 'Indication': 4530, 'Dose': 5150, 'Frequency': 4407, 'Duration': 901, 'ADE': 2055}, 'SSLIF': {'Severity': 2909, 'ADE': 4}, 'ADE': {'Severity': 282, 'ADE': 23}, 'Dose': {'Dose': 27}, 'Indication': {'Severity': 269, 'Indication': 24}, 'Severity': {'Severity': 16}, 'Duration': {'Duration': 5}, 'Route': {'Route': 7}, 

In [108]:
set(types_not_connected)

{'ADE => ADE',
 'ADE => Dose',
 'ADE => Drug',
 'ADE => Duration',
 'ADE => Frequency',
 'ADE => Indication',
 'ADE => Route',
 'ADE => SSLIF',
 'ADE => Severity',
 'Dose => ADE',
 'Dose => Dose',
 'Dose => Drug',
 'Dose => Duration',
 'Dose => Frequency',
 'Dose => Indication',
 'Dose => Route',
 'Dose => SSLIF',
 'Dose => Severity',
 'Drug => ADE',
 'Drug => Dose',
 'Drug => Drug',
 'Drug => Duration',
 'Drug => Frequency',
 'Drug => Indication',
 'Drug => Route',
 'Drug => SSLIF',
 'Drug => Severity',
 'Duration => ADE',
 'Duration => Dose',
 'Duration => Drug',
 'Duration => Duration',
 'Duration => Frequency',
 'Duration => Indication',
 'Duration => Route',
 'Duration => SSLIF',
 'Duration => Severity',
 'Frequency => ADE',
 'Frequency => Dose',
 'Frequency => Drug',
 'Frequency => Duration',
 'Frequency => Frequency',
 'Frequency => Indication',
 'Frequency => Route',
 'Frequency => SSLIF',
 'Frequency => Severity',
 'Indication => ADE',
 'Indication => Dose',
 'Indication => Dr

In [112]:
string = "# of documents: {num_docs}\nTotal # of entities: {num_entities}\nTotal # relations: {num_relations}\n# unique entity types: {num_entity_types}\n# unique relations: {num_relationship_types}\n\n".format(**info)
string += "Most common edges: \n\t{}".format('\n\t'.join([str(tup) for tup in 
                                    sorted(type_relations.items(), key=lambda x:x[1], reverse=True)]))


string += "\n\n"
string += "Most common entities: \n\t{}".format('\n\t'.join(
                    [str(tup) for tup in 
                    sorted(entity_types.items(), key=lambda x:x[1], reverse=True)]
                ))
string += "\n\n"
string += "Impossible edges: \n\t{}".format('\n\t'.join(set(types_not_connected)))
print(string)

# of documents: 876
Total # of entities: 67781
Total # relations: 23165
# unique entity types: 9
# unique relations: 17

Most common edges: 
	('Drug => Dose', 5150)
	('Drug => Indication', 4530)
	('Drug => Frequency', 4407)
	('SSLIF => Severity', 2909)
	('Drug => Route', 2544)
	('Drug => ADE', 2055)
	('Drug => Duration', 901)
	('ADE => Severity', 282)
	('Indication => Severity', 269)
	('Dose => Dose', 27)
	('Indication => Indication', 24)
	('ADE => ADE', 23)
	('Severity => Severity', 16)
	('Frequency => Frequency', 12)
	('Route => Route', 7)
	('Duration => Duration', 5)
	('SSLIF => ADE', 4)

Most common entities: 
	('Drug', 19587)
	('Dose', 5204)
	('Indication', 4847)
	('Frequency', 4431)
	('Severity', 3492)
	('SSLIF', 2913)
	('Route', 2558)
	('ADE', 2387)
	('Duration', 911)

Impossible edges: 
	Severity => Severity
	Drug => Drug
	Indication => ADE
	ADE => Drug
	Frequency => Route
	Severity => Frequency
	ADE => Severity
	SSLIF => Route
	Dose => Severity
	Duration => Indication
	ADE => 

In [113]:
with open('exploration_results.txt', 'w') as f:
    f.write(string)

In [85]:
string += "Most common edges: \n\t{}".format('\n\t'.join([' => '.join(types) + ': '+str(count) for 
                     (types, count) in 
                    sorted(entity_types.items(), key=lambda x:x[1], reverse=True)]))

defaultdict(int,
            {'ADE': 2387,
             'Dose': 5204,
             'Drug': 19587,
             'Duration': 911,
             'Frequency': 4431,
             'Indication': 4847,
             'Route': 2558,
             'SSLIF': 2913,
             'Severity': 3492})